# EvoGFuzz: An Evolutionary Approach to Grammar-Based Fuzzing

**EvoGFuzz** stands for *evolutionary grammar-based fuzzing*. This approach leverages evolutionary optimization techniques to systematically explore the space of a program's potential inputs, with a particular emphasis on identifying inputs that could lead to exceptional behavior. With a user-defined objective, EvoGFuzz can adapt and refine the input generation strategy over time, making it a powerful tool for uncovering software defects and vulnerabilities.

Efficient detection of defects and vulnerabilities hinges on the ability to automatically generate program inputs that are both valid and diverse. One common strategy is to use grammars, which provide structured and syntactically correct inputs. This approach leads to the concept of grammar-based fuzzing, where fuzzing strategies are guided by the rules defined within the grammar.

A further enhancement to this concept is probabilistic grammar-based fuzzing, where competing grammar rules are associated with probabilities that guide their application. By carefully assigning and optimizing these probabilities, we gain considerable control over the nature of the generated inputs. This enables us to direct the fuzzing process towards specific areas of interest—for example, those functions that are deemed critical, have a higher propensity for failures, or have undergone recent modifications. 

In essence, EvoGFuzz represents a potent blend of evolutionary optimization and probabilistic grammar-based fuzzing, poised to reveal hidden defects and vulnerabilities in a targeted and efficient manner.

## Fuzzing a Program

Our program under investigation is `The Calculator`. This program acts as a typical calculator, capable of evaluating not just arithmetic expressions but also trigonometric functions, such as sine, cosine, and tangent. Furthermore, it also supports the calculation of the square root of a given number.

In [1]:
import math

def calculator(inp: str) -> float:
    """
        A simple calculator function that can evaluate arithmetic expressions 
        and perform basic trigonometric functions and square root calculations.
    """
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

**Side Note:** In the `calculator`, we use Python's `eval` function, which takes a string and evaluates it as a Python expression. We provide a dictionary as the second argument to eval, mapping names to corresponding mathematical functions. This enables us to use the function names directly within the input string. 

In [2]:
# Evaluating the cosine of 2π
print(calculator('cos(6*3.141)'))

0.999993677717667


In [3]:
# Calculating the square root of 36
print(calculator('sqrt(6*6)'))

6.0


Each of these calls to the calculator will evaluate the provided string as a mathematical expression, and print the result.

Now, to find new defects, we need to introduce an oracle that tells us if the error that is triggered is something we expect or a new/unkonwn defect. The `OracleResult` is an enum with two possible values, `NO_BUG` and `BUG`. `NO_BUG` donates a passing test case and `BUG` a failing one.

We import the `OracleResult` enumerated type from the `evogfuzz` library. This is used in the oracle function to indicate the outcome of executing the 'calculator' function with a given input.

In [4]:
from debugging_framework.input.oracle import OracleResult

This is a function called **oracle**, which acts as an intermediary to handle and classify exceptions produced by the calculator function when given a certain input.

In [5]:
# Make sure you use the OracleResult from the debugging_framework library
from debugging_framework.input.oracle import OracleResult

def oracle(inp: str):
    """
    This function serves as an oracle or intermediary that catches and handles exceptions 
    generated by the 'calculator' function. The oracle function is used in the context of fuzz testing.
    It aims to determine whether an input triggers a bug in the 'calculator' function.

    Args:
        inp (str): The input string to be passed to the 'calculator' function.

    Returns:
        OracleResult: An enumerated type 'OracleResult' indicating the outcome of the function execution.
            - OracleResult.PASSING: Returned if the calculator function executes without any exception or only with CalculatorSyntaxError
            - OracleResult.FAILING: Returned if the calculator function raises a ValueError exception, indicating a potential bug.
    """
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.FAILING
    
    return OracleResult.PASSING

This **oracle** function is used in the context of fuzzing to determine the impact of various inputs on the program under test (in our case the _calculator_). When the calculator function behaves as expected (i.e., no exceptions occur), the **oracle** function returns `OracleResult.NO_BUG`. However, when the `calculator` function raises an unexpected exception, the **oracle** interprets this as a potential bug in the `calculator` and returns `OracleResult.BUG`.

We can see this in action by testing a few initial inputs:

In [6]:
initial_inputs = ['sqrt(1)', 'cos(912)', 'tan(4)']

for inp in initial_inputs:
    print(inp.ljust(20), oracle(inp))

sqrt(1)              PASSING
cos(912)             PASSING
tan(4)               PASSING


The following code represents a simple context-free grammar for our calculator function. This grammar encompasses all the potential valid inputs to the calculator, which include mathematical expressions involving square roots, trigonometric functions, and integer and decimal numbers:

In [7]:
from debugging_framework.types import Grammar
from debugging_framework.fuzzingbook.grammar import is_valid_grammar

CALCGRAMMAR: Grammar = {
    "<start>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],
    
    "<term>": ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
    
assert is_valid_grammar(CALCGRAMMAR)

The defined grammar CALCGRAMMAR provides a structured blueprint for creating various inputs for our fuzz testing. Each rule in this grammar reflects a possible valid input that our calculator function can handle. By fuzzing based on this grammar, we can systematically explore the space of valid inputs to the calculator function.

### Leveraging EvoGFuzz to Unearth New Defects

We apply our `EvoGFuzz` class to carry out fuzz testing using evolutionary grammar-based fuzzing. This is aimed at uncovering potential defects in our 'calculator' function.

To initialize our EvoGFuzz instance, we require a grammar (in our case, `CALCGRAMMAR`), an oracle function, an initial set of inputs, a fitness function, and the number of iterations to be performed in the fuzzing process.

Upon creating the `EvoGFuzz` instance, we can execute the fuzzing process. The `fuzz()` method runs the fuzzing iterations, evolving the inputs based on our fitness function, and returns a collection of inputs that lead to exceptions in the 'calculator' function.

In [8]:
from evogfuzz.evogfuzz_class import EvoGFuzz

epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10
)

Upon creating the `EvoGFuzz` instance, we can execute the fuzzing process. The `.fuzz()` method runs the fuzzing iterations, evolving the inputs based on our fitness function, and returns a collection of inputs that lead to exceptions in the 'calculator' function.

In [9]:
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 171 bug-triggering inputs!


Lastly, we can examine the inputs that resulted in exceptions. This output can provide valuable insight into potential weaknesses in the 'calculator' function that need to be addressed.

In [10]:
# print only the first 20 bug-triggering inputs
for inp in list(found_exception_inputs)[:20]:
    print(str(inp))

sqrt(-1414)
sqrt(-499.2)
sqrt(-9.4)
sqrt(-9.12)
sqrt(-44.12)
sqrt(-1949194.291929)
sqrt(-991.12)
sqrt(-29999144.11119)
sqrt(-9.91)
sqrt(-91.9912)
sqrt(-9919)
sqrt(-91.21)
sqrt(-11.2)
sqrt(-9)
sqrt(-19.9)
sqrt(-2.12)
sqrt(-21.1)
sqrt(-1191.19)
sqrt(-194.19219)
sqrt(-2.2911)


This process illustrates the power of evolutionary grammar-based fuzzing in identifying new defects within our system. By applying evolutionary algorithms to our fuzzing strategy, we can guide the search towards more defect-prone regions of the input space.

#### Analyzing and Sorting All Generated Inputs by Fitness

After the fuzzing process, you may want to examine all the generated inputs. These can be accessed using the `get_all_inputs()` method. Additionally, we can sort these inputs based on their fitness scores to gain insights into which inputs performed best according to our fitness function.

In [11]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

Now, let's print out these sorted inputs along with their respective fitness scores. Inputs with higher fitness scores will be displayed first, as these are the ones our evolutionary process deemed more likely to uncover potential defects.

In [12]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

sqrt(-499.2)                             fitness: 1
sqrt(-1949194.291929)                    fitness: 1
sqrt(-1191.19)                           fitness: 1
sqrt(-2.2911)                            fitness: 1
sqrt(-91.12999)                          fitness: 1
sqrt(-491.2)                             fitness: 1
sqrt(-41)                                fitness: 1
sqrt(-9.21)                              fitness: 1
sqrt(-49)                                fitness: 1
sqrt(-1911.1)                            fitness: 1
sqrt(-121.14)                            fitness: 1
sqrt(-11)                                fitness: 1
sqrt(-9.92)                              fitness: 1
sqrt(-9919.1)                            fitness: 1
sqrt(-22.919)                            fitness: 1
sqrt(-2.2)                               fitness: 1
sqrt(-9999.21491)                        fitness: 1
sqrt(-91211.4)                           fitness: 1
sqrt(-4)                                 fitness: 1
sqrt(-24.22)

This output provides an overview of the evolved inputs and their effectiveness in revealing potential defects, as gauged by our fitness function. It is a valuable resource for understanding the behavior of our program under various inputs and the effectiveness of our evolutionary grammar-based fuzzing approach.

In [13]:
from evogfuzz.input import Input

def fitness_function_naive(test_input: Input) -> int:
    score_structure = len(str(test_input))
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [14]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=fitness_function_naive,
    iterations=10
)

found_exception_inputs = epp.fuzz()

print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 23 bug-triggering inputs!


In [15]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

In [16]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

sqrt(-3694679786666)                     fitness: 120
sqrt(-4279698463)                        fitness: 117
sqrt(-376777969)                         fitness: 116
sqrt(-43973463)                          fitness: 115
sqrt(-9341647)                           fitness: 114
sqrt(-784286)                            fitness: 113
sqrt(-124269)                            fitness: 113
sqrt(-626642)                            fitness: 113
sqrt(-17732)                             fitness: 112
sqrt(-67636)                             fitness: 112
sqrt(-7692)                              fitness: 111
sqrt(-1878)                              fitness: 111
sqrt(-476)                               fitness: 110
sqrt(-799)                               fitness: 110
sqrt(-297)                               fitness: 110
sqrt(-997)                               fitness: 110
sqrt(-11)                                fitness: 109
sqrt(-29)                                fitness: 109
sqrt(-78)                   

In [17]:
def count_expansions(children):
    if children == []:
        return 0

    counter = 1

    for child in children:
        _, next_children = child
        counter += count_expansions(next_children)    
    
    return counter

In [18]:
from evogfuzz.input import Input

def improved_fitness_function(test_input: Input) -> float:
    _, children = test_input.tree
    number_expansions = count_expansions(children)
    lam = 100
    score_structure = (number_expansions**2)/(lam * len(str(test_input)))
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [19]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=improved_fitness_function,
    iterations=10
)

found_exception_inputs = epp.fuzz()

print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 15 bug-triggering inputs!


In [20]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

In [21]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

sqrt(-4412429142942499)                  fitness: 100.56347826086956
sqrt(-24142442492221)                    fitness: 100.48761904761905
sqrt(-4994292129422)                     fitness: 100.45
sqrt(-4429229999429)                     fitness: 100.45
sqrt(-9442194991)                        fitness: 100.33882352941177
sqrt(-4912144)                           fitness: 100.23142857142857
sqrt(-24424)                             fitness: 100.16333333333333
sqrt(-41249)                             fitness: 100.16333333333333
sqrt(-1492)                              fitness: 100.13090909090909
sqrt(-1119)                              fitness: 100.13090909090909
sqrt(-429)                               fitness: 100.1
sqrt(-94)                                fitness: 100.07111111111111
sqrt(-4)                                 fitness: 100.045
sqrt(-9)                                 fitness: 100.045
sqrt(-1)                                 fitness: 100.045
sqrt(243543243455832334948274149486

In [22]:
def calculate_score_structure(children, height):
    score = 0
    
    for child in children:
        node, next_children = child        
        score += len(next_children)**height
        score += calculate_score_structure(next_children, height+1)

    return score

In [38]:
from evogfuzz.input import Input

def sophisticated_fitness_function(test_input: Input) -> float:
    score_structure = calculate_score_structure([test_input.tree],0)
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 2**100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [39]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=sophisticated_fitness_function,
    iterations=10
)

found_exception_inputs = epp.fuzz()

print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 13 bug-triggering inputs!


In [40]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

In [41]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:    
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

sqrt(-54444154655568144758488718118189164964559) fitness: 1267650600228229410292796227623
sqrt(-11611155911971561615645846)        fitness: 1267650600228229401496971640856
sqrt(-3994589394599911553484)            fitness: 1267650600228229401496719982612
sqrt(-341911185441564)                   fitness: 1267650600228229401496703336461
sqrt(-51149416614)                       fitness: 1267650600228229401496703213577
sqrt(-95564461)                          fitness: 1267650600228229401496703206406
sqrt(-7455646)                           fitness: 1267650600228229401496703205893
sqrt(-469864)                            fitness: 1267650600228229401496703205636
sqrt(-641)                               fitness: 1267650600228229401496703205409
sqrt(-64)                                fitness: 1267650600228229401496703205392
sqrt(-9)                                 fitness: 1267650600228229401496703205383
sqrt(-1)                                 fitness: 1267650600228229401496703205383
sqrt(-7)

In [42]:
def calculate_height_and_degreeSum(children, height):
    max_height = height
    score = 0
    
    for child in children:
        node, next_children = child
        score += len(next_children)**height
        next_score, next_height = calculate_height_and_degreeSum(next_children, height+1)
        score += next_score
        if next_height > max_height:
            max_height = next_height
            
    return score, max_height

In [51]:
from evogfuzz.input import Input

def ratio_sophisticated_fitness_function(test_input: Input) -> float:
    #_, children = test_input.tree
    lam = 2**50
    degreeSums, height = calculate_height_and_degreeSum([test_input.tree],0)
    score_structure = degreeSums/(lam * height)
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [52]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=ratio_sophisticated_fitness_function,
    iterations=10
)

found_exception_inputs = epp.fuzz()

print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 118 bug-triggering inputs!


In [53]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

In [54]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:    
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

tan(-1114141942144192999112111929119149114914141211111141119141112419211944149911114299119191419111.919911912211121411419911141) fitness: 710795395733.9799
cos(-29244941191491211421442921224414294214194119911111221111441419.1199211111211411142112121119) fitness: 244.53731344706978
sqrt(-21214114411411994111412412141111191112.49) fitness: 100.00002271075581
sqrt(-1441144.11194121294212119111119121441411419) fitness: 100.00000305175783
sqrt(-14111114491949191191414411429124944) fitness: 100.00000305175782
sqrt(-9.2411424114112111949449199421999994) fitness: 100.00000156500401
sqrt(-21111412114112111412211491241941.1944999411211111194449192) fitness: 100.00000041562159
sqrt(-244449424992199114194911249414)    fitness: 100.00000010899136
sqrt(-111111112421114241941.149941119419491111141494911) fitness: 100.00000001513399
sqrt(-44419111.112299124192411941411144419) fitness: 100.00000001490119
sqrt(-11111914412112119191119114)        fitness: 100.00000000769093
sqrt(-149411119411199911111419

In [32]:
# for debugging

#from isla.derivation_tree import DerivationTree
#from isla.parser import EarleyParser

#inp = Input(DerivationTree.from_parse_tree(next(EarleyParser(CALCGRAMMAR).parse("sqrt(-13.62)"))))
#node, children = inp.tree
#score_structure = calculate_height_and_degreeSum([inp.tree],0)
#print(score_structure)

### Incorporating Custom Fitness Functions

The fitness function plays a crucial role in guiding the evolution process of our fuzzing inputs. A well-crafted fitness function can effectively direct the search towards the most promising regions of the input space.

To create your own fitness function, define a function that takes an `Input` instance and returns a float value. The return value represents the 'fitness' of the given input, with higher values indicating better fitness. Here is a simple template:

```python
from evogfuzz.input import Input

def fitness_function_XYZ(inp: Input) -> float:
    # Implement your fitness function here.
    return 0.0
```

For instance, suppose we're interested in inputs that invoke the cosine function in our calculator. We could define a fitness function `fitness_function_cos` that assigns a high fitness value to inputs containing 'cos'. (**Note that this might not be the best fitness function to find new expcetions.**)

In [33]:
from evogfuzz.input import Input

def fitness_function_cos(inp: Input) -> float:
    if 'cos' in str(inp):
        return 1.0
    else:
        return 0.0

Once your fitness function is defined, you can incorporate it into the `EvoGFuzz` instance by passing it as the `fitness_function` argument. 

In [34]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=fitness_function_cos,
    iterations=10
)

found_exception_inputs = epp.fuzz()

print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

for inp in found_exception_inputs:
    print(str(inp))

EvoGFuzz found 0 bug-triggering inputs!


This way, the evolutionary grammar-based fuzzing process is now guided by your custom fitness function, focusing more on the areas you deem critical.

#### Evaluating Inputs Based on Custom Fitness Function

When utilizing a custom fitness function, such as `fitness_function_cos` in our case, we expect inputs containing 'cos' to achieve the highest fitness scores. This is because our fitness function assigns a score of 1.0 to any input that includes 'cos'.

To confirm this behavior, we retrieve all inputs generated during the fuzzing process using the `get_all_inputs()` method and sort these inputs based on their fitness scores.

In [23]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

Let's display these sorted inputs along with their fitness scores. The inputs that contain 'cos' should appear first, demonstrating their high fitness value.

In [24]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

cos(7.26)                                fitness: 1.0
cos(19)                                  fitness: 1.0
cos(-2762)                               fitness: 1.0
cos(7.6)                                 fitness: 1.0
cos(76397.53)                            fitness: 1.0
cos(7126.3)                              fitness: 1.0
cos(757.72)                              fitness: 1.0
cos(162821.87)                           fitness: 1.0
cos(42)                                  fitness: 1.0
cos(5.2)                                 fitness: 1.0
cos(4)                                   fitness: 1.0
cos(1.412)                               fitness: 1.0
cos(42.1)                                fitness: 1.0
cos(224.24)                              fitness: 1.0
cos(-6)                                  fitness: 1.0
cos(7.71)                                fitness: 1.0
cos(942.41)                              fitness: 1.0
cos(-9.2)                                fitness: 1.0
cos(7.1)                    

The resulting output validates the effectiveness of our custom fitness function. It shows how we can guide the evolutionary grammar-based fuzzing process towards specific regions of the input space, thereby facilitating targeted exploration and bug discovery.